In [3]:
import sys
sys.path.append('../')

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
%reload_ext autoreload

In [13]:
from scripts.detect_voids import get_neightbours
from packages.yolo_predict import YOLO_Pred
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import cv2

In [9]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/data/first_training/weights/bestnoft.onnx', '/home/cremerf/FinalProject/data/config_blmodel.yaml')

In [10]:
df_predictions = yolo.predictions('/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [10]:
dict_of_neightbours = get_neightbours(img_path='/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [11]:
dict_of_neightbours[0]['572']

[571, 564, 273, 277, 275, 599, 600]

In [13]:
def bb_intersection_over_union(df_predictions: pd.DataFrame, index_key:int, index_neightbour:int) -> float:

    a = index_key
    xA1 = df_predictions.loc[a][0] - df_predictions.loc[a][2]/2
    yA1 = df_predictions.loc[a][1] + df_predictions.loc[a][3]/2
    xA2 = df_predictions.loc[a][0] + df_predictions.loc[a][2]/2
    yA2 = df_predictions.loc[a][1] - df_predictions.loc[a][3]/2
    boxA = [xA1, yA1, xA2, yA2 ]

    b = index_neightbour # indices vecinos detectados que estan en el diccionario
    xB1 = df_predictions.loc[b][0] - df_predictions.loc[b][2]/2
    yB1 = df_predictions.loc[b][1] + df_predictions.loc[b][3]/2
    xB2 = df_predictions.loc[b][0] + df_predictions.loc[b][2]/2
    yB2 = df_predictions.loc[b][1] - df_predictions.loc[b][3]/2
    boxB = [xB1, yB1,xB2, yB2 ]
    
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])


    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
# para left w = -w y h = 0
# para right w = +w y h = 0
# para top w = 0 y h = + h

In [14]:
df_predictions_test = df_predictions.copy()

In [15]:
df_predictions_test.loc[1260]

Center_X    194.691193
Center_Y    139.134720
Width        22.782619
Height       90.207382
Name: 1260, dtype: float64

In [91]:
df_predictions_test['Height_over_image'] =  df_predictions_test.Center_Y + (df_predictions_test.Height/2)

In [92]:
df_predictions_test[df_predictions_test['Height_over_image'] == df_predictions_test['Height_over_image'].max()]

,Center_X,Center_Y,Width,Height,Height_over_image
1250,18.964607,563.583618,26.012243,46.716412,586.941824


In [16]:
df_predictions.loc[572][2]

23.51447868347168

In [17]:
image = cv2.imread(filename='/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [18]:
image.shape[0:2]

(3264, 2448)

In [19]:
# para el diccionario de neightbours 

for index_a, index_b in dict_of_neightbours[0].items():
    print(f'This bb {index_a} has neightbours')
    h_image, w_image = image.shape[0:2] # obtengo limites de la imagen
    w_index_a = df_predictions.loc[int(index_a)][2]
    xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 - w_index_a
    yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
    xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 - w_index_a
    yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2
    boxA = [xA1, yA1, xA2, yA2]
    if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
        for item in index_b:
            trigger = False 
            print(f'Neightbour bound box: {item}')
            xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
            yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
            xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
            yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
            boxB = [xB1, yB1,xB2, yB2]

            xA = max(boxA[0], boxB[0])
            yA = max(boxA[1], boxB[1])
            xB = min(boxA[2], boxB[2])
            yB = min(boxA[3], boxB[3])

            interArea = (xB - xA) * (yB - yA)

            boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
            boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

            iou = interArea / float(boxAArea + boxBArea - interArea)
            if iou <= 0.05:
                trigger = True
            else:
                trigger = False
        if trigger:
            print(f'A la izquierda del bb: {index_a} hay un espacio VACIO {boxA}')

This bb 572 has neightbours
Neightbour bound box: 571
Neightbour bound box: 564
Neightbour bound box: 273
Neightbour bound box: 277
Neightbour bound box: 275
Neightbour bound box: 599
Neightbour bound box: 600
A la izquierda del bb: 572 hay un espacio VACIO [186.2997236251831, 273.0568199157715, 209.81420230865479, 341.10559463500977]
This bb 1059 has neightbours
Neightbour bound box: 1054
Neightbour bound box: 825
Neightbour bound box: 1055
A la izquierda del bb: 1059 hay un espacio VACIO [173.40342807769775, 197.78594970703125, 198.07180309295654, 268.36993408203125]
This bb 581 has neightbours
Neightbour bound box: 576
Neightbour bound box: 577
Neightbour bound box: 580
Neightbour bound box: 579
Neightbour bound box: 317
Neightbour bound box: 20
A la izquierda del bb: 581 hay un espacio VACIO [290.0626678466797, 276.2918586730957, 306.35304260253906, 337.2691764831543]
This bb 585 has neightbours
Neightbour bound box: 581
Neightbour bound box: 583
Neightbour bound box: 580
Neightbou

In [130]:
df_predictions.loc[1059]

Center_X    210.405991
Center_Y    233.077942
Width        24.668375
Height       70.583984
Name: 1059, dtype: float64

In [129]:
df_predictions.loc[1059][2]

24.66837501525879

In [123]:
xA1 = df_predictions.loc[1059][0] - df_predictions.loc[1059][2]/2 - w_index_a
yA1 = df_predictions.loc[1059][1] - df_predictions.loc[1059][3]/2
xA2 = df_predictions.loc[1059][0] + df_predictions.loc[1059][2]/2 - w_index_a
yA2 = df_predictions.loc[1059][1] + df_predictions.loc[1059][3]/2

In [125]:
xA1, yA1, xA2, yA2

(184.86670780181885,
 268.36993408203125,
 209.53508281707764,
 197.78594970703125)

In [134]:
dict_of_neightbours[0]['572']

[571, 564, 273, 277, 275, 599, 600]

In [135]:
df_predictions.loc[571]

Center_X    199.205215
Center_Y    306.348145
Width        22.828110
Height       64.707008
Name: 571, dtype: float64

In [136]:
for item in dict_of_neightbours[0]['216']:
    print(values)

1028
1027
791


In [ ]:
dict_of_neightbours[0]

In [ ]:
for index_a, index_b in dict_of_neightbours[0].items():
    h_image, w_image = image.shape[0:2] # obtengo limites de la imagen
    w_index_a = df_predictions.loc[int(index_a)][2]
    xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 - w_index_a
    yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
    xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 - w_index_a
    yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2
    boxA = [xA1, yA1, xA2, yA2]
    print(f'Evaluating {index_a}...')
    if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
            trigger = True
            for item in index_b: 
                xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
                yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
                xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
                yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
                boxB = [xB1, yB1,xB2, yB2]

                xA = max(boxA[0], boxB[0])
                yA = max(boxA[1], boxB[1])
                xB = min(boxA[2], boxB[2])
                yB = min(boxA[3], boxB[3])

                interArea = (xB - xA) * (yB - yA)

                boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
                boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

                iou = interArea / float(boxAArea + boxBArea - interArea)
                trigger = trigger and (iou < 0.1)
            #print(f'Trigger: {trigger}')
            if trigger == False:
                print(f'No hay espacio vacio a la izquierda de {index_a}')
            else:
                print(f'A la izquierda de {index_a} hay espacio vacio')

In [ ]:
dict_of_neightbours

In [93]:
from operator import add, sub, mul
def math(x, y, function):
      return function(x, y)

In [96]:
df_predictions.loc[216]

Center_X    168.405960
Center_Y    140.233566
Width        33.719307
Height       74.962959
Name: 216, dtype: float64

In [99]:
X_center_A = df_predictions.loc[216][0] - df_predictions.loc[216][2]  
Y_center_A = df_predictions.loc[216][1]

(134.68665313720703, 106.5142593383789)

In [ ]:
X_center_A = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]  
Y_center = df_predictions.loc[int(index_a)][1]  

In [103]:
def search_voids_bb_neightbours_left(df_predictions: pd.DataFrame, dict_of_neightbours: dict):
    
    list_of_voids = []
    for index_a, index_b in dict_of_neightbours.items():

        first_lists = []
        h_image, w_image = image.shape[0:2] # obtengo limites de la imagen
        w_index_a = df_predictions.loc[int(index_a)][2]
        h_index_a = df_predictions.loc[int(index_a)][3]
        
        xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 - w_index_a
        yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
        xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 - w_index_a
        yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2
        boxA = [xA1, yA1, xA2, yA2]

        void_number = 1
        void_text = f'Void number {void_number}'        
        X_center_A = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]  #X_center - Width
        Y_center_A = df_predictions.loc[int(index_a)][1]  - df_predictions.loc[int(index_a)][2]  #Y_center - Width

        print(f'Evaluating {index_a}...')
        if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
                trigger = True
                for item in index_b:
                    xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
                    yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
                    xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
                    yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
                    boxB = [xB1, yB1,xB2, yB2]

                    xA = max(boxA[0], boxB[0])
                    yA = max(boxA[1], boxB[1])
                    xB = min(boxA[2], boxB[2])
                    yB = min(boxA[3], boxB[3])

                    interArea = (xB - xA) * (yB - yA)

                    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
                    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

                    iou = interArea / float(boxAArea + boxBArea - interArea)
                    trigger = trigger and (iou < 0.1)
                #print(f'Trigger: {trigger}')
                if trigger == False:
                    pass
                    #print(f'No hay espacio vacio a la izquierda de {index_a}')
                else:
                    print(f'A la izquierda de {index_a} hay espacio vacio')
                    
                    list_of_voids.append(boxA)

    return list_of_voids

In [ ]:
# izquierda (-1)
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 - k * w_index_a
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 - k * w_index_a
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2

In [ ]:
# derecha (+1)
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 + k * w_index_a
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 + k * w_index_a
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2

In [ ]:
# arriba 
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2 - h_index_a
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2 - h_index_a

In [ ]:
# arriba 
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 + k * w_index_a # Para izquierda: (-1) / Para derecha: (1) / Para arriba: 0
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2 + k * h_index_a # Para izquierda: 0 / Para derecha: 0 / Para arriba: -1
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 + k * w_index_a # Para izquierda: (-1) / Para derecha: (1) / Para arriba: 0
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2 + K * h_index_a # Para izquierda: 0 / Para derecha: 0 / Para arriba: -1

In [102]:
list_of_voids = search_voids_bb_neightbours(df_predictions= df_predictions, dict_of_neightbours= dict_of_neightbours[0])

Evaluating 572...
Evaluating 1059...
Evaluating 581...
Evaluating 585...
Evaluating 1203...
Evaluating 576...
Evaluating 432...
Evaluating 1067...
Evaluating 577...
Evaluating 583...
Evaluating 574...
Evaluating 1172...
Evaluating 580...
Evaluating 1205...
Evaluating 1054...
Evaluating 1297...
Evaluating 496...
Evaluating 571...
Evaluating 494...
Evaluating 586...
Evaluating 1294...
Evaluating 431...
Evaluating 1235...
Evaluating 1237...
Evaluating 834...
Evaluating 89...
Evaluating 1060...
Evaluating 1201...
Evaluating 459...
Evaluating 425...
Evaluating 457...
Evaluating 579...
Evaluating 564...
Evaluating 696...
Evaluating 825...
Evaluating 393...
Evaluating 430...
Evaluating 230...
Evaluating 1174...
Evaluating 399...
Evaluating 66...
Evaluating 391...
Evaluating 587...
Evaluating 273...
Evaluating 489...
Evaluating 1053...
Evaluating 561...
Evaluating 143...
Evaluating 422...
Evaluating 225...
Evaluating 277...
Evaluating 185...
Evaluating 270...
Evaluating 1132...
Evaluating 1260

In [170]:
def search_voids_bb_neightbours(df_predictions: pd.DataFrame, dict_of_neightbours: dict, neightbour:str=('left','right','up')):
    
    list_of_voids = []

    k = 0
    z = 0
    if neightbour == 'left':
        k = -1
        z = 0
    elif neightbour == 'right':
        k = 1
        z = 0
    else:
        k = 0
        z = -1
    
    for index_a, index_b in dict_of_neightbours.items():

        h_image, w_image = image.shape[0:2] # obtengo limites de la imagen
        w_index_a = df_predictions.loc[int(index_a)][2]
        h_index_a = df_predictions.loc[int(index_a)][3]


        
        # Virtual bounding box to evaluate neightbours 
        xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 + (k * w_index_a) # Para izquierda: (-1) / Para derecha: 1 / Para arriba: 0
        yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2 + (z * h_index_a) # Para izquierda: 0 / Para derecha: 0 / Para arriba: (-1)
        xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 + (k * w_index_a) # Para izquierda: (-1) / Para derecha: 1 / Para arriba: 0
        yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2 + (z * h_index_a) # Para izquierda: 0 / Para derecha: 0 / Para arriba: -1
        boxA = [xA1, yA1, xA2, yA2]


        X_center_A = df_predictions.loc[int(index_a)][0] - k * df_predictions.loc[int(index_a)][2]  # Left X_center - Width  // Right X_center + Width
        Y_center_A = df_predictions.loc[int(index_a)][1]  - k * df_predictions.loc[int(index_a)][2]  # Left Y_center - Width // Right Y_center + Width

        print(f'Evaluating {index_a}...')
        

        # Limits of the image
        if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
                trigger = True
                void_number = 0

                # Iterate over each neightbour: neightbour vs virtual bounding box
                for item in index_b:
                    
                    first_list = []
                    xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
                    yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
                    xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
                    yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
                    boxB = [xB1, yB1,xB2, yB2]

                    xA = max(boxA[0], boxB[0])
                    yA = max(boxA[1], boxB[1])
                    xB = min(boxA[2], boxB[2])
                    yB = min(boxA[3], boxB[3])

                    interArea = (xB - xA) * (yB - yA)

                    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
                    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

                    iou = interArea / float(boxAArea + boxBArea - interArea)
                    trigger = trigger and (iou < 0.1)
                #print(f'Trigger: {trigger}')

                if trigger == False:
                    pass
                    #print(f'No hay espacio vacio a la izquierda de {index_a}')
                else:
                    print(f'A la izquierda de {index_a} hay espacio vacio')
                    void_number += 1
                    void_text = f'{neightbour} void #{void_number}'
                    first_list.append(index_a)
                    first_list.append(void_text)
                    first_list.append(X_center_A)
                    first_list.append(Y_center_A)
                    list_of_voids.append(first_list)
                


    return list_of_voids

In [171]:
list_of_voids = search_voids_bb_neightbours(df_predictions=df_predictions, dict_of_neightbours= dict_of_neightbours[0], neightbour='left')

-1
Evaluating 572...
Evaluating 1059...
Evaluating 581...
Evaluating 585...
Evaluating 1203...
Evaluating 576...
Evaluating 432...
Evaluating 1067...
Evaluating 577...
Evaluating 583...
Evaluating 574...
Evaluating 1172...
Evaluating 580...
Evaluating 1205...
Evaluating 1054...
Evaluating 1297...
Evaluating 496...
Evaluating 571...
Evaluating 494...
Evaluating 586...
Evaluating 1294...
Evaluating 431...
Evaluating 1235...
Evaluating 1237...
Evaluating 834...
Evaluating 89...
Evaluating 1060...
Evaluating 1201...
Evaluating 459...
Evaluating 425...
Evaluating 457...
Evaluating 579...
Evaluating 564...
Evaluating 696...
Evaluating 825...
Evaluating 393...
Evaluating 430...
Evaluating 230...
Evaluating 1174...
Evaluating 399...
Evaluating 66...
Evaluating 391...
Evaluating 587...
Evaluating 273...
Evaluating 489...
Evaluating 1053...
Evaluating 561...
Evaluating 143...
Evaluating 422...
Evaluating 225...
Evaluating 277...
Evaluating 185...
Evaluating 270...
Evaluating 1132...
Evaluating 1

In [ ]:
list_of_voids

In [18]:
import multiprocessing
from functools import partial

In [11]:
img_path = '/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg'

In [14]:
list_of_dicts = get_neightbours(img_path= img_path, df_predictions= df_predictions)

In [17]:
pool = multiprocessing.Pool()

In [27]:
df_predictions.index

Int64Index([ 572, 1059,  581,  585, 1203,  576,  432, 1067,  577,  583,
            ...
             221, 1198, 1119,   94, 1250, 1145,  712,  310,  600,   43],
           dtype='int64', length=161)

In [28]:
def get_neightbours_test(df_predictions:pd.DataFrame, index) -> list:
    for index_bb in df_predictions.loc[index]:
            for index in df_predictions.index_bb:
                print(index)

In [24]:
df_predictions.loc[572]

X_center    221.571442
Y_center    307.081207
Width        23.514479
Height       68.048775
Name: 572, dtype: float64

In [36]:
def get_neightbours_test(img_path:str, df_predictions:pd.DataFrame, neightbour:str=('left','right','up')) -> list:

    dict_l_n = {} # neigh left
    dict_r_n = {} # neigh right
    dict_h_n= {}  # alone right

    for index_bb in df_predictions.index:
        print(index_bb)
        list_of_neightbours_l = []
        list_of_neightbours_r = []
        list_of_neightbours_h = [] 
        list_of_alones_l = []
        list_of_alones_r = []
        list_of_alones_h=[]
        for index in df_predictions.index:
            print(index)

            X_center_0 = df_predictions.loc[index_bb][0]
            Y_center_0 = df_predictions.loc[index_bb][1]
            Width_0 = df_predictions.loc[index_bb][2]
            height_0 = df_predictions.loc[index_bb][3]
            threshold_x_0 = 0.8 * Width_0
            threshold_y_0 = 0.8 * height_0

            X_center_neightbour = df_predictions.loc[index][0]
            Y_center_neightbour = df_predictions.loc[index][1]
            a=4
            k=1.2

        if neightbour == 'left':
            ### Neightbor Left
            x_min_l = X_center_0 - Width_0 - (a*threshold_x_0)
            x_max_l = X_center_0 - Width_0 + (k*threshold_x_0)

            y_min_l = Y_center_0 - (k*threshold_y_0)
            y_max_l = Y_center_0 + (k*threshold_y_0)

        elif neightbour == 'right': 
            ### Neightbor Right
            x_min_r = X_center_0 + Width_0 - (k*threshold_x_0)
            x_max_r = X_center_0 + Width_0 + (a*threshold_x_0)

            y_min_r = Y_center_0 - (k*threshold_y_0)
            y_max_r = Y_center_0 + (k*threshold_y_0)

        else:
            ### Neightbor High
            x_min_h = X_center_0 - (k*threshold_x_0)
            x_max_h = X_center_0 + (2*k*threshold_x_0)

            y_min_h = Y_center_0 + height_0 - (0.8*threshold_y_0)
            y_max_h = Y_center_0 + height_0 + (2*a*threshold_y_0)
            
            ### Search neig left
            if (x_min_l < X_center_neightbour < x_max_l and y_min_l < Y_center_neightbour < y_max_l )  :
                list_of_neightbours_l.append([index])
                try:
                    dict_l_n[str(index_bb)].append(index)
                except KeyError:
                    dict_l_n[str(index_bb)]=[]
                    dict_l_n[str(index_bb)].append(index)
            else:
                list_of_alones_l.append([index])

            if (x_min_r < X_center_neightbour < x_max_r and y_min_r < Y_center_neightbour < y_max_r ):
                list_of_neightbours_r.append([index])
                try:
                    dict_r_n[str(index_bb)].append(index)
                except KeyError:
                    dict_r_n[str(index_bb)]=[]
                    dict_r_n[str(index_bb)].append(index)
            else:
                list_of_alones_r.append([index_bb, index])
                
            if (x_min_h < X_center_neightbour < x_max_h and y_min_h < Y_center_neightbour < y_max_h ):
                list_of_neightbours_h.append([index])
                try:
                    dict_h_n[str(index_bb)].append(index)
                except KeyError:
                    dict_h_n[str(index_bb)]=[]
                    dict_h_n[str(index_bb)].append(index)
            else:
                list_of_alones_h.append([index_bb, index])
    print('listo primer loop')

    return [dict_l_n, dict_r_n, dict_h_n]

In [ ]:
for i in df_predictions.index:
    print(type(i))

In [57]:
df_predictions.loc[572].values

array([221.57144165, 307.08120728,  23.51447868,  68.04877472])

In [76]:
df_predictions.loc[df_predictions.index == 572].values

array([[221.57144165, 307.08120728,  23.51447868,  68.04877472]])

In [ ]:
for 572 in df_predictions.index:
    for index in df_predictions.index:
        print(index_bb)
        print(index)


In [ ]:
def test(df_predictions, index_it):
    for i in df_predictions.loc[df_predictions.index == index_it].values:
        X_center_0 = df_predictions.loc[index_it][0]
        Y_center_0 = df_predictions.loc[index_it][1]
        Width_0 = df_predictions.loc[index_it][2]
        height_0 = df_predictions.loc[index_it][3]
        threshold_x_0 = 0.8 * Width_0
        threshold_y_0 = 0.8 * height_0

        X_center_neightbour = df_predictions.loc[index][0]
        Y_center_neightbour = df_predictions.loc[index][1]

        #Width_0 = df_predictions.loc[572][2]
        #X_center_neightbour = df_predictions.loc[index][0]
        #height_0 = df_predictions.loc[572][3]


In [144]:
df_predictions

,X_center,Y_center,Width,Height
572,221.571442,307.081207,23.514479,68.048775
1059,210.405991,233.077942,24.668375,70.583984
581,314.498230,306.780518,16.290375,60.977318
585,343.513489,306.846436,16.417421,60.421963
1203,148.945190,518.730042,27.151506,27.399454
...,...,...,...,...
1145,119.181030,465.013733,24.931517,14.736507
712,446.199127,483.145142,42.025547,15.644592
310,427.420837,317.954681,10.442470,28.763885
600,167.968338,316.750610,21.088881,58.980614


In [102]:
def get_neightbours_test(df_predictions:pd.DataFrame, index_it:int, neightbour:str=('left','right','up')) -> list:

    dict_l_n = {} # neigh left
    dict_r_n = {} # neigh right
    dict_h_n= {}  # alone right

    # puedo prescindir de este for
    for i in df_predictions.loc[df_predictions.index == index_it].values:
        list_of_neightbours_l = []
        list_of_neightbours_r = []
        list_of_neightbours_h = [] 
        list_of_alones_l = []
        list_of_alones_r = []
        list_of_alones_h=[]
        #cambiar index_bb por index_it
        # creo que puedo prescindir de todos los for
        for index in df_predictions.index:
            X_center_0 = i[0]
            Y_center_0 = i[1]
            Width_0 = i[2]
            height_0 = i[3]
            threshold_x_0 = 0.8 * Width_0
            threshold_y_0 = 0.8 * height_0

            X_center_neightbour = df_predictions.loc[index][0]
            Y_center_neightbour = df_predictions.loc[index][1]
            a=4
            k=1.2
            if neightbour == 'left':
            ### Neightbor Left
                x_min_l = X_center_0 - Width_0 - (a*threshold_x_0)
                x_max_l = X_center_0 - Width_0 + (k*threshold_x_0)

                y_min_l = Y_center_0 - (k*threshold_y_0)
                y_max_l = Y_center_0 + (k*threshold_y_0)
            elif neightbour == 'right':
            ### Neightbor Right
                x_min_r = X_center_0 + Width_0 - (k*threshold_x_0)
                x_max_r = X_center_0 + Width_0 + (a*threshold_x_0)

                y_min_r = Y_center_0 - (k*threshold_y_0)
                y_max_r = Y_center_0 + (k*threshold_y_0)
            else:
            ### Neightbor High
                x_min_h = X_center_0 - (k*threshold_x_0)
                x_max_h = X_center_0 + (2*k*threshold_x_0)

                y_min_h = Y_center_0 + height_0 - (0.8*threshold_y_0)
                y_max_h = Y_center_0 + height_0 + (2*a*threshold_y_0)
            
            ### Search neig left
            if (x_min_l < X_center_neightbour < x_max_l and y_min_l < Y_center_neightbour < y_max_l )  :
                list_of_neightbours_l.append([index])
                try:
                    dict_l_n[str(index_it)].append(index)
                except KeyError:
                    dict_l_n[str(index_it)]=[]
                    dict_l_n[str(index_it)].append(index)
            else:
                list_of_alones_l.append([index])

            if (x_min_r < X_center_neightbour < x_max_r and y_min_r < Y_center_neightbour < y_max_r ):
                list_of_neightbours_r.append([index])
                try:
                    dict_r_n[str(index_it)].append(index)
                except KeyError:
                    dict_r_n[str(index_it)]=[]
                    dict_r_n[str(index_it)].append(index)
            else:
                list_of_alones_r.append([index_it, index])
                
            if (x_min_h < X_center_neightbour < x_max_h and y_min_h < Y_center_neightbour < y_max_h ):
                list_of_neightbours_h.append([index])
                try:
                    dict_h_n[str(index_it)].append(index)
                except KeyError:
                    dict_h_n[str(index_it)]=[]
                    dict_h_n[str(index_it)].append(index)
            else:
                list_of_alones_h.append([index_it, index])
    print('listo primer loop')

    return [dict_l_n, dict_r_n, dict_h_n]

In [107]:
pool = multiprocessing.Pool()
func = partial(get_neightbours_test, df_predictions)
test1 = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

listo primer looplisto primer looplisto primer loop


listo primer loop
listo primer looplisto primer loop
listo primer loop
listo primer loop

listo primer looplisto primer loop
listo primer loop

listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer looplisto primer loop

listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer looplisto primer loop

listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer looplisto primer loop

listo prim

In [143]:
test1[0]

[{'572': [571, 564, 273, 277, 275, 599, 600]},
 {'572': [581, 576, 577, 574, 580, 579, 317, 20]},
 {'572': [1172, 496, 459, 457, 1174, 399, 1132, 364, 498, 1110, 1150, 333]}]

In [124]:
# The idea is to pool a bunch of chunks by each thread
with ThreadPoolExecutor() as executor:
    list_of_dicts = list(executor.map(lambda index: get_neightbours_test(df_predictions, index), df_predictions.index))

listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer looplisto primer loop
listo primer loop
listo primer loop
listo primer loop

listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer looplisto primer loop
listo primer loop
listo primer loop
listo primer looplisto primer loop
listo primer loop
listo primer loop
listo primer loop


listo primer loop
listo primer loop
listo primer loop
listo primer looplisto primer loop
listo primer loop

listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer loop
listo primer looplisto primer loop
listo primer looplisto primer loop
listo primer loop

listo primer looplisto primer loop
listo primer loop
listo primer loop
listo primer loop

listo primer loop
listo primer loop
listo primer loop
listo primer looplisto primer loop
listo primer loop

listo primer loop
listo prime

In [128]:
list_of_dicts[1][2]

{'1059': [572,
  576,
  574,
  1172,
  496,
  571,
  459,
  457,
  1174,
  399,
  1132,
  364,
  498,
  1110,
  1150,
  333]}

In [145]:
def get_neightbours(df_predictions:pd.DataFrame) -> list:

    dict_l_n = {} # neigh left
    dict_r_n = {} # neigh right
    dict_h_n= {}  # alone right

    for index_bb in df_predictions.index:
        list_of_neightbours_l = []
        list_of_neightbours_r = []
        list_of_neightbours_h = [] 
        list_of_alones_l = []
        list_of_alones_r = []
        list_of_alones_h=[]
        for index in df_predictions.index:
            X_center_0 = df_predictions.loc[index_bb][0]
            Y_center_0 = df_predictions.loc[index_bb][1]
            Width_0 = df_predictions.loc[index_bb][2]
            height_0 = df_predictions.loc[index_bb][3]
            threshold_x_0 = 0.8 * Width_0
            threshold_y_0 = 0.8 * height_0

            X_center_neightbour = df_predictions.loc[index][0]
            Y_center_neightbour = df_predictions.loc[index][1]
            a=4
            k=1.2

            ### Neightbor Left
            x_min_l = X_center_0 - Width_0 - (a*threshold_x_0)
            x_max_l = X_center_0 - Width_0 + (k*threshold_x_0)

            y_min_l = Y_center_0 - (k*threshold_y_0)
            y_max_l = Y_center_0 + (k*threshold_y_0)
            
            ### Neightbor Right
            x_min_r = X_center_0 + Width_0 - (k*threshold_x_0)
            x_max_r = X_center_0 + Width_0 + (a*threshold_x_0)

            y_min_r = Y_center_0 - (k*threshold_y_0)
            y_max_r = Y_center_0 + (k*threshold_y_0)
            
            ### Neightbor High
            x_min_h = X_center_0 - (k*threshold_x_0)
            x_max_h = X_center_0 + (2*k*threshold_x_0)

            y_min_h = Y_center_0 + height_0 - (0.8*threshold_y_0)
            y_max_h = Y_center_0 + height_0 + (2*a*threshold_y_0)
            
            ### Search neig left
            if (x_min_l < X_center_neightbour < x_max_l and y_min_l < Y_center_neightbour < y_max_l )  :
                list_of_neightbours_l.append([index])
                try:
                    dict_l_n[str(index_bb)].append(index)
                except KeyError:
                    dict_l_n[str(index_bb)]=[]
                    dict_l_n[str(index_bb)].append(index)
                
            else:
                list_of_alones_l.append([index])
            if (x_min_r < X_center_neightbour < x_max_r and y_min_r < Y_center_neightbour < y_max_r ):
                list_of_neightbours_r.append([index])
                try:
                    dict_r_n[str(index_bb)].append(index)
                except KeyError:
                    dict_r_n[str(index_bb)]=[]
                    dict_r_n[str(index_bb)].append(index)
            else:
                list_of_alones_r.append([index_bb, index])
                
            # Search neig hight    
            if (x_min_h < X_center_neightbour < x_max_h and y_min_h < Y_center_neightbour < y_max_h):
                list_of_neightbours_h.append([index])
                try:
                    dict_h_n[str(index_bb)].append(index)
                except KeyError:
                    dict_h_n[str(index_bb)]=[]
                    dict_h_n[str(index_bb)].append(index)
            else:
                list_of_alones_h.append([index_bb, index])

    return [dict_l_n, dict_r_n, dict_h_n]

In [147]:
df_predictions.shape

(161, 4)

In [146]:
test2 = get_neightbours(df_predictions=df_predictions)